In [1]:
import cv2
import numpy as np
import tensorflow as tf

# Modeli yükle
model = tf.keras.models.load_model("model.keras")

# Etiketlerin tanımlanması
class_labels = ["glass", "paper", "cardboard", "plastic", "metal", "trash"]

# Kamerayı başlat
cap = cv2.VideoCapture(0)

# Tespit için güven sınırı
CONFIDENCE_THRESHOLD = 0.8

def detect_objects():
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Kameradan görüntü alınamadı.")
            break

        # OpenCV ile görüntüyü yeniden boyutlandır ve işleme hazır hale getir
        input_image = cv2.resize(frame, (224, 224))  # MobilNetV2 için varsayılan boyut
        input_image = np.expand_dims(input_image, axis=0)  # Batch boyutunu ekle
        input_image = input_image / 255.0  # Normalize et

        # Model ile tahmin yap
        predictions = model.predict(input_image)
        confidence = np.max(predictions)
        predicted_class_index = np.argmax(predictions)  # En yüksek tahmin indeksini al
        predicted_class_name = class_labels[predicted_class_index]  # İndeksi sınıf adına dönüştür

        # Görüntü üzerine bilgi ekle
        height, width, _ = frame.shape

        # Merkezde bir kutu oluştur (örnek boyutlarla)
        box_center_x, box_center_y = width // 2, height // 2
        box_width, box_height = 150, 150

        # Kutunun sol üst ve sağ alt köşe koordinatlarını hesapla
        top_left_x = box_center_x - box_width // 2
        top_left_y = box_center_y - box_height // 2
        bottom_right_x = box_center_x + box_width // 2
        bottom_right_y = box_center_y + box_height // 2

        # Güven sınırını geçen sınıf bulunursa kutu çiz ve bilgiyi yaz
        if confidence > CONFIDENCE_THRESHOLD:
            # Kutuyu çiz
            cv2.rectangle(frame, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (0, 255, 0), 2)
            # Sınıf adını ve güveni yaz
            cv2.putText(frame, f"Class: {predicted_class_name}, Confidence: {confidence:.2f}",
                        (top_left_x, top_left_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Güncel görüntüyü göster
        cv2.imshow("Real-time Detection", frame)

        # Çıkmak için 'q' tuşuna bas
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Kamerayı serbest bırak ve tüm pencereleri kapat
    cap.release()
    cv2.destroyAllWindows()

# Nesne tespiti fonksiyonunu çalıştır
detect_objects()


1/1 [==============================] - 0s 74ms/step
